Project which I did for a summer school course on machine learning applied to speech technology, at the University of Eastern Finland.

The goal of this task is to try to determine whether subjects have Parkinson’s Disease, using a neural network with input features extracted from speech recordings. The recordings involve the subjects saying “Ahhh” into their iPhone microphones for several seconds.

In [ ]:
# Check that the correct version of Python is running (3.5)
import sys
sys.version

In [ ]:
import numpy as np
import scipy
import librosa
import librosa.display
from IPython.display import Audio
import scipy.io.wavfile
import keras
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import regularizers
from keras import optimizers
import pickle
from sklearn import metrics
% matplotlib inline

# Read & Extract Info from CSV File

In [ ]:
## Put in path to pd-data folder here
datapath = "~/uef/data/pd-data/"

In [ ]:
# Read CSV data file
raw_data = pd.read_csv('~/uef/data/pd-data/dataset.csv', delimiter=";")
#print(raw_data)

In [ ]:
# Convert panda object columns into np arrays
ids = np.array(raw_data['id'])
ages = np.array(raw_data['age'])
times = np.array(raw_data['time'])
pds_string = np.array(raw_data['pd'])
evals = np.array(raw_data['eval'])
files = np.array(raw_data['file'])

In [ ]:
# Convert string entries for the PD column into 1's and 0's
pds = np.zeros(pds_string.shape)
for i in range(0,len(pds_string)):
    if pds_string[i] == 'True':
        pds[i] = 1
    elif pds_string[i] == 'False':
        pds[i] = 0

In [ ]:
# Since I only have audio files up to entry #nn, select only the first nn entries
n_temp = nn #(put value in here)

ids = ids[:n_temp]
ages = ages[:n_temp]
times = times[:n_temp]
pds = pds[:n_temp]
evals = evals[:n_temp]
files = files[:n_temp]

# Extract Features

In [ ]:
##################################################
#           Feature Extraction                   #
##################################################

# Some of these functions were adapted from the Front-End Feature Extraction notebook used in class.

def read_file(file_name, sample_rate):
    X_orig, sample_rate = librosa.load(file_name, sr=sample_rate)
    return X_orig, sample_rate

def extract_feature(X, sample_rate_new):
    #X = librosa.effects.trim(X_orig, top_db=60)
    stft = np.abs(librosa.stft(X))
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate_new, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate_new)
    #mel = librosa.feature.melspectrogram(X, sr=sample_rate)
    #contrast = librosa.feature.spectral_contrast(S=stft, sr=sample_rate)
    #tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=X)
    #return mfccs,chroma,mel,contrast,tonnetz
    #return mfccs, chroma, mel, contrast, tonnetz
    return mfccs, chroma, zero_crossing_rate
    #return zero_crossing_rate

def squared_derivative(matrix, step):
    ## Calculate the "derivative":
    ## the difference between a feature at one time step and the previous time step.
    ## Square it because I'm interested in the magnitude, not sign
    ## Step is the time scale over which the "derivative" is taken:
    ## We take feature[t] - feature[t-step].
    
    n_rows = matrix.shape[0]
    n_frames = matrix.shape[1]
    dev_sq = np.zeros([n_rows, n_frames-step])
    for i in range(0,n_rows):
        for j in range(step,n_frames):
            diff = matrix[i,j] - matrix[i, j-step]
            dev_sq[i,j-step] = diff**2
    return dev_sq

def mean_stddev(matrix, n_ignore):
    ## Calculate mean and standard deviation over all time steps (axis 1).
    ## n_ignore is if we want to ignore certain time steps, say the first, for whatever reason
    ## Matrix is 2D: n_features x n_time_steps
    
    means = np.mean(matrix[:,n_ignore:], axis=1)
    stddevs = np.std(matrix[:,n_ignore:], axis=1)
    
    return means, stddevs

def mean_stddev_list(thelist, n_ignore):
    ## For a list of arrays corresponding to features over time steps for different sound files,
    ## Calculate mean and standard deviation over all time steps
    ## The list is a list over sound files
    ## Each element of the list is a 2D array: n_features x n_time_steps
    
    meansarr = np.array([])
    stddevsarr = np.array([])
    for i in range(0, len(thelist)):
        means, stddevs = mean_stddev(thelist[i], n_ignore)
        if (i == 0):
            meansarr = np.copy(means)
            stddevsarr = np.copy(stddevs)
        else:
            meansarr = np.vstack((meansarr, means))
            stddevsarr = np.vstack((stddevsarr, stddevs))
        
    return meansarr, stddevsarr

def squared_derivative_list(thelist, step):
    ## For a list of arrays corresponding to features over time steps for different sound files,
    ## Calculate squared derivative
    
    dev_sqarr = []
    for ele in thelist:
        dev_sq = squared_derivative(ele, step)
        dev_sqarr.append(dev_sq)

    return dev_sqarr

In [ ]:
# Extract basic features: MFCCs, Chroma, Zero Crossing Rate
sr = 16000
mfccsarr = []
chromaarr = []
zerocrossingarr = []
i_count_arr = []
for i_count in range(0, len(files)):
    file = files[i_count]
    if (np.mod(i_count,100) == 0):
        print(i_count)
    X_orig, sample_rate = read_file(datapath+file, sr)
    if (len(X_orig) == 0):
        print("Zero length")
        continue
    X = (librosa.effects.trim(X_orig, top_db=35))[0] # Remove silence regions (35 dB below maximum in file)
    mfccs, chroma, zero_crossing_rate = extract_feature(X, sr) # Extract features
    
    mfccsarr.append(mfccs)
    chromaarr.append(chroma)
    zerocrossingarr.append(zero_crossing_rate)
    i_count_arr.append(i_count) # i_count keeps track of which indexes have had features successfully extracted
                                # i.e. those which don't have empty sound files
    i_count = i_count + 1

In [ ]:
# Save basic features in files
with open("./saved_objects/icountarr", "wb") as fp:
    pickle.dump(i_count_arr, fp)
with open("./saved_objects/mfccsarr", "wb") as fp:
    pickle.dump(mfccsarr, fp)
with open("./saved_objects/chromaarr", "wb") as fp:
    pickle.dump(chromaarr, fp)
with open("./saved_objects/zerocrossingarr", "wb") as fp:
    pickle.dump(zerocrossingarr, fp)

In [ ]:
# Load basic features from files
with open("./saved_objects/icountarr", "rb") as fp:
    i_count_arr = pickle.load(fp)
with open("./saved_objects/mfccsarr", "rb") as fp:
    mfccsarr = pickle.load(fp)
with open("./saved_objects/chromaarr", "rb") as fp:
    chromaarr = pickle.load(fp)
with open("./saved_objects/zerocrossingarr", "rb") as fp:
    zerocrossingarr = pickle.load(fp)

In [ ]:
# For the np arrays containing info from the CSV file,
# We need to remove those entries which had zero-length sound files (no features extracted)

ids_new = np.array([])
ages_new = np.array([])
times_new = np.array([])
pds_new = np.array([])
evals_new = np.array([])
files_new = np.array([])
for i in range(0, len(i_count_arr)):
    index = i_count_arr[i]
    
    ids_new = np.append(ids_new, ids[index])
    ages_new = np.append(ages_new, ages[index])
    times_new = np.append(times_new, times[index])
    pds_new = np.append(pds_new, pds[index])
    evals_new = np.append(evals_new, evals[index])
    files_new = np.append(files_new, files[index])

ids = ids_new
ages = ages_new
times = times_new
pds = pds_new
evals = evals_new
files = files_new

# Make Calculations on Features

In [ ]:
# Compute mean and standard deviation of features
mfccs_meansarr, mfccs_stddevsarr = mean_stddev_list(mfccsarr, 0)
chroma_meansarr, chroma_stddevsarr = mean_stddev_list(chromaarr, 0)
zerocrossing_meansarr, zerocrossing_stddevsarr = mean_stddev_list(zerocrossingarr, 0)

In [ ]:
# Compute derivatives, and mean and stddev of derivatives, of features
step = 1
mfccs_devrarr = squared_derivative_list(mfccsarr, step)
chroma_devrarr = squared_derivative_list(chromaarr, step)
zerocrossing_devrarr = squared_derivative_list(zerocrossingarr, step)
mfccs_devr_meanarr, mfccs_devr_stddevarr = mean_stddev_list(mfccs_devrarr, 0)
chroma_devr_meanarr, chroma_devr_stddevarr = mean_stddev_list(chroma_devrarr, 0)
zerocrossing_devr_meanarr, zerocrossing_devr_stddevarr = mean_stddev_list(zerocrossing_devrarr, 0)

In [ ]:
# Compute derivatives with a different step size, and mean and stddev of derivatives, of features
step = 2
mfccs_devrarr2 = squared_derivative_list(mfccsarr, step)
chroma_devrarr2 = squared_derivative_list(chromaarr, step)
zerocrossing_devrarr2 = squared_derivative_list(zerocrossingarr, step)
mfccs_devr_meanarr2, mfccs_devr_stddevarr2 = mean_stddev_list(mfccs_devrarr2, 0)
chroma_devr_meanarr2, chroma_devr_stddevarr2 = mean_stddev_list(chroma_devrarr2, 0)
zerocrossing_devr_meanarr2, zerocrossing_devr_stddevarr2 = mean_stddev_list(zerocrossing_devrarr2, 0)

In [ ]:
# Reduce number of MFCCs, if I wish
newnum = 13
mfccs_meansarr_red = mfccs_meansarr[:,0:newnum]
mfccs_stddevsarr_red = mfccs_stddevsarr[:,0:newnum]
mfccs_devr_meanarr_red = mfccs_devr_meanarr[:,0:newnum]
mfccs_devr_stddevarr_red = mfccs_devr_stddevarr[:,0:newnum]

# Prepare Features for Feeding into NN

In [ ]:
# Put all relevant features into a single array to feed into NN:
featurearr = np.array([])
for i in range(0, mfccs_meansarr.shape[0]):
    featurelist = np.concatenate((mfccs_stddevsarr[i,:], \
                                  mfccs_devr_meanarr[i,:], \
                                  chroma_devr_meanarr[i,:], zerocrossing_devr_meanarr[i,:],
                                  mfccs_meansarr[i,:]), 0)
    if (i==0):
        featurearr = np.copy(featurelist)
    else:
        featurearr = np.vstack((featurearr, featurelist))

In [ ]:
# Check shape of feature array
featurearr.shape

In [ ]:
# Normalise data by subtracting mean and dividing by standard deviation
epsilon = 1e-8 # to prevent division by zero
featurearr_norm = np.zeros_like(featurearr)
for i in range (0,featurearr.shape[1]):
    featurearr_norm[:,i] = (featurearr[:,i]-np.mean(featurearr[:,i]))/(np.std(featurearr[:,i] + epsilon))

# Split Data into Training, Validation, Evaluation Sets

In [ ]:
# Split into evaluation and training-validation sets
# Note: Evaluation set is the one for which the "eval" column says "True"
# I will use the rest for experimentation, by splitting it into training and validation sets

Y_eval = pds[np.where(evals == True)]
X_eval = featurearr_norm[np.where(evals == True)]
Y_trainval = pds[np.where(evals == False)]
X_trainval = featurearr_norm[np.where(evals == False)]
speakers_trainval = ids[np.where(evals == False)] # Speaker ids who are not in eval set

In [ ]:
# Split non-evaluation data again into training and validation sets
# Ensure that the same speaker doesn't go into both sets (i.e. each speaker is only in one set)

train_val_ratio = 1.5 # 60% training data; 40% validation data
epsilon = 1e-6 # To prevent division by zero
train_id_list = np.array([]) #Speaker ids assigned to training set
val_id_list = np.array([]) #Speaker ids assigned to validation set
X_train = np.array([])
Y_train = np.array([])
X_validation = np.array([])
Y_validation = np.array([])
istrain = True

for i in range(0, len(speakers_trainval)):
    speaker = speakers_trainval[i]
    #print(speaker in train_id_list)
    if speaker in train_id_list:
        # If speaker is already assigned to training set, all his subsequent recordings are too
        istrain = True
    elif speaker in val_id_list:
        # If speaker is already assigned to validation set, all his subsequent recordings are too
        istrain = False
    else:
        # If we have more training points than we wanted, assign next speaker to validation set
        if (len(Y_train)/(len(Y_validation) + epsilon) > 1.5):
            istrain = False
        # Otherwise assign to training set
        else:
            istrain = True
    if istrain:
        if (len(X_train) == 0):
            X_train = X_trainval[i,:]
        else:
            X_train = np.vstack((X_train, X_trainval[i,:]))
        Y_train = np.append(Y_train, Y_trainval[i])
        train_id_list = np.append(train_id_list, speaker)
    else:
        if (len(X_validation) == 0):
            X_validation = X_trainval[i,:]
        else:
            X_validation = np.vstack((X_validation, X_trainval[i,:]))
        Y_validation = np.append(Y_validation, Y_trainval[i])
        val_id_list = np.append(val_id_list, speaker)

In [ ]:
# Check training and validation set sizes, indeed they are at a ratio of 1.5
print(X_train.shape)
print(X_validation.shape)
print(Y_train.shape)
print(Y_validation.shape)

In [ ]:
# Check that the ratio of PD vs. non-PD in the training set, validation set, and all data is about the same
Y_pd_train = Y_train[np.where(Y_train == 1)]
print(Y_pd_train.shape)
Y_nopd_train = Y_train[np.where(Y_train == 0)]
print(Y_nopd_train.shape)
Y_pd_val = Y_validation[np.where(Y_validation == 1)]
print(Y_pd_val.shape)
Y_nopd_val = Y_validation[np.where(Y_validation == 0)]
print(Y_nopd_val.shape)
haspd = pds[np.where(pds == 1)]
nopd = pds[np.where(pds == 0)]
print(haspd.shape)
print(nopd.shape)

# Make Model

In [ ]:
# This cell contains code that I copied/adapted from Anssi's eval_script.py, 
# so that I can use it to calculated EER and AUROC without writing things into files

import os
import sys
os.environ['SIDEKIT'] = 'theano=false,libsvm=false'

import numpy as np
import sidekit
import sklearn.metrics

def pmiss_pfs_eer(tar, non_tar):
    minDCF, Pmiss, Pfa, prbep, eer = sidekit.bosaris.detplot.fast_minDCF(
        tar, non_tar, np.log(0.001), normalize=True)
    return eer

def compute_auroc(tar, non_tar):
    scores = np.hstack((tar,non_tar))
    classes = np.hstack((np.ones(tar.shape), np.zeros(non_tar.shape)))
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(classes, scores)
    auroc = sklearn.metrics.auc(fpr, tpr)
    return auroc

def compute_eer_and_auroc(pos, neg):
    eer = pmiss_pfs_eer(pos,neg)
    auroc = compute_auroc(pos, neg)
    
    print("EER: %f" % eer)
    print("AUROC: %f" % auroc)
    
    return eer, auroc

In [ ]:
## Define the model

def define_model(X_train):
    model = keras.models.Sequential()

    model.add(Dense(units=64, input_dim=X_train.shape[1], activation="tanh"))
    model.add(Dropout(0.5))
    model.add(Dense(units=32, activation="tanh", kernel_regularizer=regularizers.l1(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(units=16, activation="tanh", kernel_regularizer=regularizers.l1(0.001)))
    model.add(Dropout(0.1))
    #model.add(Dense(units=8, activation="tanh", kernel_regularizer=regularizers.l1(0.001)))
    #model.add(Dropout(0.1))
    #model.add(Dense(units=4, activation="tanh", kernel_regularizer=regularizers.l1(0.01)))

    #model.add(Dense(units=4, activation="tanh"))
    #model.add(Dropout(0.25))
    #kernel_regularizer=regularizers.l2(0.02)
    #model.add(Dense(units=128, activation="relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(units=1, activation="sigmoid", kernel_regularizer=regularizers.l1(0.01)))
    #model.add(Dense(units=1, input_dim=X_train.shape[1], activation="sigmoid", kernel_regularizer=regularizers.l2(1.0)))

    opt = optimizers.RMSprop(lr=0.001)

    model.compile(loss='binary_crossentropy',
                 optimizer=opt,
                 metrics=['accuracy'])
    
    return model

In [ ]:
## A function that runs the model training several times, 
## and calculates the average EER and AUROC on the validation set
## Since these values are somewhat dependent on the initialisation conditions
## Use this to select model parameters, feature sets

def model_auroc(X_train, Y_train, X_validation, Y_validation, n_times):
    eer_total = 0
    auroc_total = 0
    for i in range(0, n_times):
        model = define_model(X_train)
        model.fit(X_train, Y_train, epochs=70, batch_size=256, validation_data = (X_validation, Y_validation))
        
        Y_predictions = model.predict(X_train)
        Y_validation_predictions = model.predict(X_validation)
        positives = np.ravel(Y_validation_predictions[np.where(Y_validation == 1)])
        negatives = np.ravel(Y_validation_predictions[np.where(Y_validation == 0)])
        eer, auroc = compute_eer_and_auroc(positives, negatives)
        eer_total = eer_total + eer
        auroc_total = auroc_total + auroc
        
    eer_average = eer_total/n_times
    auroc_average = auroc_total/n_times
    return eer_average, auroc_average, model

In [ ]:
eer_average, auroc_average, model = model_auroc(X_train, Y_train, X_validation, Y_validation, 10)

In [ ]:
print("Average EER: %f" % eer_average)
print("Average AUROC: %f" % auroc_average)

# Calculate Some Metrics

Just run the training process once for this section

In [ ]:
## If I just want to run the training process once, I use this

model = define_model(X_train)
model.fit(X_train, Y_train, epochs=70, batch_size=256, validation_data = (X_validation, Y_validation))

In [ ]:
Y_predictions = model.predict(X_train)
Y_validation_predictions = model.predict(X_validation)
positives = np.ravel(Y_validation_predictions[np.where(Y_validation == 1)])
negatives = np.ravel(Y_validation_predictions[np.where(Y_validation == 0)])
compute_eer_and_auroc(positives, negatives)

In [ ]:
# Compare AUROC for training set and validation set
print(metrics.roc_auc_score(Y_train, Y_predictions))
print(metrics.roc_auc_score(Y_validation, Y_validation_predictions))

In [ ]:
## Use a threshold to convert prediction score to binary classifier
## So that I can calculate confusion matrix
def cont_to_binary(arr):
    threshold = 0.5
    arr_binary = np.zeros_like(arr)
    for i in range(0, len(arr)):
        if arr[i] < threshold:
            arr_binary[i] = 0
        else:
            arr_binary[i] = 1
    return arr_binary

In [ ]:
Y_pred_class = cont_to_binary(Y_predictions)
Y_validation_pred_class = cont_to_binary(Y_validation_predictions)

In [ ]:
## Confusion matrix for training set 
print(metrics.classification_report(Y_train, Y_pred_class))
print(metrics.confusion_matrix(Y_train, Y_pred_class))

In [ ]:
## Confusion matrix for validation set
print(metrics.classification_report(Y_validation, Y_validation_pred_class))
print(metrics.confusion_matrix(Y_validation, Y_validation_pred_class))

# Obtain Evaluation Set Score

In [ ]:
## A function that runs the model training several times, 
## and calculates the average EER and AUROC on the evaluation set
## Use this to calculate final evaluation score

def model_auroc(X_train, Y_train, X_eval, Y_eval, n_times):
    eer_total = 0
    auroc_total = 0
    for i in range(0, n_times):
        model = define_model(X_train)
        model.fit(X_train, Y_train, epochs=70, batch_size=256)
        
        Y_predictions = model.predict(X_train)
        Y_eval_predictions = model.predict(X_eval)
        positives = np.ravel(Y_eval_predictions[np.where(Y_eval == 1)])
        negatives = np.ravel(Y_eval_predictions[np.where(Y_eval == 0)])
        eer, auroc = compute_eer_and_auroc(positives, negatives)
        eer_total = eer_total + eer
        auroc_total = auroc_total + auroc
        
    eer_average = eer_total/n_times
    auroc_average = auroc_total/n_times
    return eer_average, auroc_average, model

In [ ]:
eer_eval, auroc_eval, model = model_auroc(X_train, Y_train, X_eval, Y_eval, 10)

In [ ]:
print("Eval EER: %f" % eer_eval)
print("Eval AUROC: %f" % auroc_eval)

# Section for Checking

In [ ]:
## Some testing to ensure that files are read correctly, etc.

In [ ]:
## Determine threshold at which to remove silence
## By looking at plots
[X_orig, sample_rate] = librosa.load(datapath+"dataset_samples/file.wav", sr=16000)
X = (librosa.effects.trim(X_orig, top_db=35))[0]

In [ ]:
## Plot of original waveform
librosa.display.waveplot(X_orig,sr=sample_rate)

In [ ]:
X_orig.shape

In [ ]:
X.shape

In [ ]:
## Plot of trimmed waveform
librosa.display.waveplot(X,sr=sample_rate)

In [ ]:
## Try extracting feature from one file
X_orig, sample_rate = read_file(datapath+'dataset_samples/file.wav', 16000)
X = (librosa.effects.trim(X_orig, top_db=35))[0]
mfccs, chroma, zero_crossing_rate = extract_feature(X, sample_rate)

In [ ]:
## Try playing audio
[y,fs] = librosa.load(datapath+"dataset_samples/file.wav", sr=16000)
Audio(data=y, rate=16000)

# Run Anssi's Baseline Code on my Training/Validation Sets

In [ ]:
## Because Anssi's Baseline Code used the Eval set as the validation set, and the rest as training sets
## Also because I am missing about a quarter of the sound files
## We have different datasets, hence I wanted to calculate the baseline using his code but my dataset

In [ ]:
## Fivenum calculator copied from Anssi's baseline code
def fivenum(x, axis):
    """ Computes five number summary along given axis
    
    x -- Input array
    axis -- Axis along which five number summary should be computed.
    Returns: Array where given axis is replaced with five number summary
    """
    mi = x.min(axis=axis)
    first_q = np.percentile(x, 25, axis=axis)
    median = np.median(x, axis=axis)
    third_q = np.percentile(x, 75, axis=axis)
    ma = x.max(axis=axis)
    return np.stack([mi, first_q, median, third_q, ma], axis=axis)

In [ ]:
## Calculate fivenum on the MFCCs that I extracted
mfccs5num = np.array([])
for i in range(0, len(mfccsarr)):
    mf = (fivenum(mfccsarr[i], 1)).ravel()
    if (i==0):
        mfccs5num = mf
    else:
        mfccs5num = np.vstack((mfccs5num, mf))

In [ ]:
# Put features into a single array to feed into NN
featurearrtest = np.array([])
for i in range(0, mfccs_meansarr.shape[0]):
    featurelisttest = np.copy(mfccs5num[i,:])
    if (i==0):
        featurearrtest = np.copy(featurelisttest)
    else:
        featurearrtest = np.vstack((featurearrtest, featurelisttest))

In [ ]:
# Normalise data by subtracting mean and dividing by standard deviation
epsilon = 1e-8 # to prevent division by zero
featurearr_normtest = np.zeros_like(featurearrtest)
for i in range (0,featurearrtest.shape[1]):
    featurearr_normtest[:,i] = (featurearrtest[:,i]-np.mean(featurearrtest[:,i]))/(np.std(featurearrtest[:,i] + epsilon))

In [ ]:
# Split into evaluation and training-validation sets
# Note: Evaluation set is the one for which the "eval" column says "True"
# I will use the rest for experimentation, by splitting it into training and validation sets

Y_evaltest = pds[np.where(evals == True)]
X_evaltest = featurearr_normtest[np.where(evals == True)]
Y_trainvaltest = pds[np.where(evals == False)]
X_trainvaltest = featurearr_normtest[np.where(evals == False)]
speakers_trainvaltest = ids[np.where(evals == False)] # Speaker ids who are not in eval set

In [ ]:
# Split non-evaluation data again into training and validation sets
# Ensure that the same speaker doesn't go into both sets (i.e. each speaker is only in one set)

train_val_ratio = 1.5 # 60% training data; 40% validation data
epsilon = 1e-6 # To prevent division by zero
train_id_list = np.array([]) #Speaker ids assigned to training set
val_id_list = np.array([]) #Speaker ids assigned to validation set
X_traintest = np.array([])
Y_traintest = np.array([])
X_validationtest = np.array([])
Y_validationtest = np.array([])
istrain = True

for i in range(0, len(speakers_trainvaltest)):
    speaker = speakers_trainvaltest[i]
    #print(speaker in train_id_list)
    if speaker in train_id_list:
        # If speaker is already assigned to training set, all his subsequent recordings are too
        istrain = True
    elif speaker in val_id_list:
        # If speaker is already assigned to validation set, all his subsequent recordings are too
        istrain = False
    else:
        # If we have more training points than we wanted, assign next speaker to validation set
        if (len(Y_traintest)/(len(Y_validationtest) + epsilon) > 1.5):
            istrain = False
        # Otherwise assign to training set
        else:
            istrain = True
    if istrain:
        if (len(X_traintest) == 0):
            X_traintest = X_trainvaltest[i,:]
        else:
            X_traintest = np.vstack((X_traintest, X_trainvaltest[i,:]))
        Y_traintest = np.append(Y_traintest, Y_trainvaltest[i])
        train_id_list = np.append(train_id_list, speaker)
    else:
        if (len(X_validationtest) == 0):
            X_validationtest = X_trainvaltest[i,:]
        else:
            X_validationtest = np.vstack((X_validationtest, X_trainvaltest[i,:]))
        Y_validationtest = np.append(Y_validationtest, Y_trainvaltest[i])
        val_id_list = np.append(val_id_list, speaker)

In [ ]:
## Train Logisic Regression
from sklearn.linear_model import LogisticRegression
modeltest = LogisticRegression()
modeltest.fit(X_trainvaltest, Y_trainvaltest)

In [ ]:
## Evaluate baseline model
Y_predictionstest = modeltest.predict(X_trainvaltest)
Y_validation_predictionstest = modeltest.predict(X_evaltest)
positivestest = np.ravel(Y_validation_predictionstest[np.where(Y_evaltest == 1)])
negativestest = np.ravel(Y_validation_predictionstest[np.where(Y_evaltest == 0)])
compute_eer_and_auroc(positivestest, negativestest)

This is lower than Anssi's reported result, probably because of the smaller training set I used.